In [119]:
#imports and setup
import pandas as pd
import os
import easymoney as em
from easymoney.money import EasyPeasy
from scipy import interpolate

ep = EasyPeasy()
years = ['1979', '1989', '1999', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

In [114]:
# Helper functions
def take_last_5(id):
    return id[-5:]

def make_num(val):
    if (type(val) == str):
        val = val.replace(',', '').replace('$', '')
        if val.isdigit():
            return float(val)
        else:
            return 0
    return val

In [115]:
# Load up income data
base_dir = '../datasets/raw/income/county/'
filenames = sorted(os.listdir(base_dir))
income_data_frames = []
for filename in filenames:
    df = pd.read_csv(base_dir + filename, encoding='latin-1', )
    income_data_frames.append(df)

In [116]:
# Shifting and merging columns/values around to get a convenient table at the end
income_data_frames[1] = income_data_frames[1].rename(index=str, columns=({ 'GEO.id2' : 'FIPS', 'GEO.display-label' : 'county', 'VD02' : '1999'}))
right = income_data_frames[1][['GEO.id', 'FIPS', 'county', '1999']]
result = pd.merge(left, right, on='county')

offset=2
for idx, df in enumerate(income_data_frames[offset:]):
    pos = offset + idx
    year = filenames[pos][:-4]
    if year == '2009':
        key = 'HC01_EST_VC12'
    else:
        key = 'HC01_EST_VC13'
    
    df_year = df.rename(index=str, columns=({
        'GEO.id2' : 'FIPS',
        key : year
    }))
    df_year = df_year[['FIPS', year]]
    result = pd.merge(result, df_year, on='FIPS')

result['FIPS'] = result['GEO.id'].apply(take_last_5)
current_income = result[['county', 'FIPS', *years]]
current_income.head()

,county,FIPS,1979,1989,1999,2009,2010,2011,2012,2013,2014,2015
0,"Autauga County, Alabama",01001,"16,524","28,337",41953,51463,53255,53899,53773,53682,52475,51281.0
1,"Baldwin County, Alabama",01003,"14,614","25,712",40174,48918,50147,51321,50706,50221,50183,50254.0
2,"Barbour County, Alabama",01005,"10,058","19,389",25080,32537,33219,34041,31889,32911,35634,32964.0
3,"Bibb County, Alabama",01007,"12,299","19,775",31402,41236,41770,40506,36824,36447,37984,38678.0
4,"Blount County, Alabama",01009,"12,657","22,382",35054,45406,45549,45404,45192,44145,44409,45813.0


In [117]:
## Normalize income to 2015 dollars
normalized_income = current_income.copy()

for year in years[:-1]:
    normalized_income[year] = current_income[year].apply(lambda val: ep.normalize(amount=make_num(val), region = "USA", from_year=int(year), to_year=2015))

#remove counties with a 0 (only 2 in this set)
normalized_income = normalized_income[~normalized_income.eq(0).any(1)]
normalized_income.head()

,county,FIPS,1979,1989,1999,2009,2010,2011,2012,2013,2014,2015
0,"Autauga County, Alabama",01001,47391.27,47579.42,52423.18,49930.17,50835.08,49875.33,48749.93,47964.84,46137.92,51281.0
1,"Baldwin County, Alabama",01003,41913.34,43171.90,50200.20,47460.97,47868.31,47489.79,45969.43,44872.43,44122.71,50254.0
2,"Barbour County, Alabama",01005,28846.61,32555.22,31339.20,31567.88,31709.52,31499.77,28910.17,29405.95,31330.70,32964.0
3,"Bibb County, Alabama",01007,35273.86,33203.34,39238.97,40007.78,39871.96,37482.15,33384.18,32565.37,33396.91,38678.0
4,"Blount County, Alabama",01009,36300.61,37580.64,43802.41,44053.58,43479.24,42014.50,40970.51,39443.53,39046.00,45813.0


In [126]:
# Interpolate income for missing years

full_income_data_set = {}
years_to_interp_between = ['1979','1989','1999','2009']

def get_values(row):
    values = {}
    for year in years_to_interp_between:
        values[year] = row[year]
    return values

def interpolate_vals(row):
    fips = row['FIPS']
    full_income_data_set[fips] = {}
    values = get_values(row)
    years = sorted(list(values.keys()))
    years_to_interp = range(1979, 2009)
    y = []
    for year in years:
        y.append(values[year])
    interpolator = interpolate.PchipInterpolator(years, y)
    for year in years_to_interp:
        full_income_data_set[fips][str(year)] = interpolator(year)

#interpolate values
normalized_income.apply(interpolate_vals, axis=1)
interp_income = pd.DataFrame.from_dict(full_income_data_set).transpose()
interp_income.index.names = ['FIPS']
cols_to_use = interp_income.columns.difference(normalized_income.columns)
full_income = pd.merge(normalized_income, interp_income[cols_to_use], on='FIPS')
year_cols = range(1979, 2016)
year_cols = sorted(list(map(lambda y: str(y), year_cols)))
full_income_sorted = full_income[['county', 'FIPS', *year_cols]]
for col in cols_to_use:
    full_income_sorted[col] = full_income_sorted[col].astype(float).round(2)

full_income_sorted.head()

,county,FIPS,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,"Autauga County, Alabama",01001,47391.27,47393.28,47399.25,47409.09,47422.72,47440.07,47461.03,47485.53,47513.49,47544.81,47579.42,47744.39,48129.54,48678.92,49336.58,50046.58,50752.95,51399.75,51931.02,52290.81,52423.18,52408.83,52361.07,52272.86,52137.13,51946.85,51694.95,51374.39,50978.10,50499.05,49930.17,50835.08,49875.33,48749.93,47964.84,46137.92,51281.0
1,"Baldwin County, Alabama",01003,41913.34,41929.37,41975.92,42050.69,42151.41,42275.77,42421.47,42586.23,42767.75,42963.74,43171.90,43541.61,44176.10,45003.83,45953.28,46952.90,47931.18,48816.58,49537.57,50022.62,50200.20,50192.11,50159.26,50088.78,49967.80,49783.46,49522.89,49173.22,48721.59,48155.13,47460.97,47868.31,47489.79,45969.43,44872.43,44122.71,50254.0
2,"Barbour County, Alabama",01005,28846.61,29450.30,30022.18,30554.80,31040.65,31472.28,31842.20,32142.93,32366.99,32506.92,32555.22,32521.17,32428.75,32292.56,32127.18,31947.21,31767.24,31601.86,31465.67,31373.25,31339.20,31339.43,31341.03,31345.37,31353.84,31367.79,31388.59,31417.64,31456.28,31505.91,31567.88,31709.52,31499.77,28910.17,29405.95,31330.70,32964.0
3,"Bibb County, Alabama",01007,35273.86,34719.87,34274.71,33926.46,33663.24,33473.15,33344.30,33264.79,33222.72,33206.20,33203.34,33360.06,33787.40,34421.11,35196.95,36050.67,36918.03,37734.78,38436.69,38959.50,39238.97,39370.97,39493.50,39605.52,39705.99,39793.86,39868.09,39927.64,39971.47,39998.53,40007.78,39871.96,37482.15,33384.18,32565.37,33396.91,38678.0
4,"Blount County, Alabama",01009,36300.61,36317.34,36365.79,36443.33,36547.35,36675.22,36824.32,36992.04,37175.74,37372.82,37580.64,37922.49,38484.03,39206.24,40030.10,40896.57,41746.65,42521.29,43161.49,43608.20,43802.41,43848.55,43890.34,43927.64,43960.35,43988.35,44011.52,44029.75,44042.91,44050.89,44053.58,43479.24,42014.50,40970.51,39443.53,39046.00,45813.0


In [127]:
## export
full_income_sorted.to_csv('../datasets/transformed/county_hh_median_income.csv')